# Etape 3.2 : Analyse des correlations

**Livrables** :
- Ce notebook `07_analyse_correlations.ipynb`
- Matrice de correlation exportee `output/07_matrice_correlation.csv`
- Synthese des insights (format markdown) `output/07_rapport_insight.md`

---
---

## Import

In [1]:
import sys
import os
from pathlib import Path
import psutil
import time
from datetime import datetime
import pandas as pd

---

## (optionnel) Enregistrement de la date de la dernière execution de ce notebook

In [2]:
print(f"- Date de la dernière execution de ce notebook : {datetime.now().strftime('%d/%m/%Y %H:%M:%S')} (FR)")

- Date de la dernière execution de ce notebook : 20/02/2026 20:33:14 (FR)


---

## (Optionnel) Mesure du temps de traitement global pour ce script - enregistrement de l'heure de début + estimation instantanée des ressources machine libres

In [3]:
## Heure de début
start_time_07 = time.time()

## Machine: current available RAM (in GB)
ram_available_07 = psutil.virtual_memory().available / (1024**3)

## Machine: current available CPU
logical = psutil.cpu_count()
physical = psutil.cpu_count(logical=False) or logical

cpu_used = psutil.cpu_percent(interval=2)
cpu_available_pct_07 = 100 - cpu_used

available_logical_07 = logical * cpu_available_pct_07 / 100
available_physical_07 = physical * cpu_available_pct_07 / 100

## Show available resources
print(f"- Current machine RAM available : {ram_available_07:.2f} GB")
print(f"- Current machine CPU available : {cpu_available_pct_07:.2f}%")
print(f"    Approx logical cores free  : {available_logical_07:.1f}")
print(f"    Approx physical cores free : {available_physical_07:.1f}")

- Current machine RAM available : 10.77 GB
- Current machine CPU available : 94.10%
    Approx logical cores free  : 15.1
    Approx physical cores free : 7.5


---

## Chemins des données

In [4]:
# ==============================================================================================================
#                                                   OUTPUTS
# ==============================================================================================================
OUT_DIR = (Path.cwd() / ".." / "output").resolve()
OUT_RAP_INSIGHT_MD = os.path.join(OUT_DIR, "07_rapport_insight.md")
OUT_MATR_COR_CSV = os.path.join(OUT_DIR, "07_matrice_correlation.csv")

# ==============================================================================================================
#                                                   INPUTS
# ==============================================================================================================
IN_DIR = (Path.cwd() / ".." / "data").resolve()
IN_CONSO_ENRICHIE_CSV =  os.path.join(OUT_DIR, "05_consommations_enrichies.csv")

# ==============================================================================================================
#                                                    OTHERS
# ==============================================================================================================
TMP_DIR = (Path.cwd() / ".." / "my_tmp").resolve()
TMP_FILE_CSV = TMP_DIR / "tmp_07_resources.txt" # Enregistrer les metrics pour ce script

---

## Chargement des données

In [5]:
df_conso = pd.read_csv(IN_CONSO_ENRICHIE_CSV)

## Affichage de quelques infos
print("df_conso :")
print(f"    - Shape: {df_conso.shape}")
print(f"    - Colonnes: {df_conso.columns.tolist()}")
print()
# Info sur les types
print("    - Infos sur les types : ")
df_conso.info()
print()
## Appercu des donnees
print("    - Appercu des donnees : ")
df_conso.head()

df_conso :
    - Shape: (991, 31)
    - Colonnes: ['batiment_id', 'heure', 'consommation_moyenne', 'unite', 'date', 'type_energie', 'ts_h', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen', 'temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh', 'precipitation_mm', 'jour', 'mois', 'saison', 'jour_de_semaine', 'date_debut', 'date_fin', 'tarif_unitaire', 'cout_financier', 'conso_par_occupant', 'conso_par_m2', 'IPE', 'ecart_moyenne_categorie']

    - Infos sur les types : 
<class 'pandas.DataFrame'>
RangeIndex: 991 entries, 0 to 990
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   batiment_id              991 non-null    str    
 1   heure                    991 non-null    int64  
 2   consommation_moyenne     991 non-null    float64
 3   unite                    991 non-null    str    
 4   date                  

,batiment_id,heure,consommation_moyenne,unite,date,type_energie,ts_h,nom,type,commune,...,saison,jour_de_semaine,date_debut,date_fin,tarif_unitaire,cout_financier,conso_par_occupant,conso_par_m2,IPE,ecart_moyenne_categorie
0,BAT0043,22,280.88,m3,2023-01-01,eau,2023-01-01 22:00:00,Piscine Bordeaux 43,piscine,Bordeaux,...,Hiver,6.0,2023-01-01,2023-12-31,3.50,983.080,1.898,0.123,0.123,-401.254
1,BAT0100,17,4.52,m3,2023-01-01,eau,2023-01-01 17:00:00,Mediatheque Rennes 100,mediatheque,Rennes,...,Hiver,6.0,2023-01-01,2023-12-31,3.50,15.820,0.037,0.005,0.005,-92.701
2,BAT0006,3,5.50,kWh,2023-01-01,gaz,2023-01-01 03:00:00,Mairie Paris 6,mairie,Paris,...,Hiver,6.0,2023-01-01,2023-06-30,0.09,0.495,0.087,0.005,0.005,-43.286
3,BAT0126,10,126.88,kWh,2023-01-01,gaz,2023-01-01 10:00:00,Ecole Le Havre 126,ecole,Le Havre,...,Hiver,6.0,2023-01-01,2023-06-30,0.09,11.419,0.375,0.075,0.075,7.068
4,BAT0035,20,3.97,m3,2023-01-02,eau,2023-01-02 20:00:00,Mairie Toulouse 35,mairie,Toulouse,...,NaN,NaN,2023-01-01,2023-12-31,3.50,13.895,0.046,0.005,0.005,-44.816


---

## Calculer la matrice de correlation entre :
  - Consommations (electricite, gaz, eau)
  - Variables meteo (temperature, humidite, rayonnement, vent)
  - Caracteristiques batiments (surface, nb occupants, annee construction)

In [6]:
# --- Construire une table pivot des consommations par énergie
pivot_conso = (
    df_conso
    .pivot_table(
        index=["batiment_id","ts_h"],
        columns="type_energie",
        values="consommation_moyenne",
        aggfunc="mean"
    )
    .reset_index()
)

# --- Ajouter météo + caractéristiques bâtiments
features = df_conso[[
    "batiment_id","ts_h",
    "temperature_c","humidite_pct",
    "rayonnement_solaire_wm2","vitesse_vent_kmh",
    "surface_m2","nb_occupants_moyen","annee_construction"
]].drop_duplicates()

df_corr = pivot_conso.merge(features, on=["batiment_id","ts_h"], how="left")

# --- Matrice de corrélation globale
matrice_corr = df_corr.corr(numeric_only=True).round(2)

matrice_corr

,eau,electricite,gaz,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,surface_m2,nb_occupants_moyen,annee_construction
eau,1.00,NaN,NaN,-0.05,0.03,0.03,-0.04,0.66,0.36,-0.18
electricite,NaN,1.00,NaN,-0.11,-0.04,-0.04,0.01,0.61,0.30,-0.08
gaz,NaN,NaN,1.00,-0.10,-0.04,-0.04,0.02,0.59,0.29,-0.14
temperature_c,-0.05,-0.11,-0.10,1.00,-0.01,-0.02,0.02,0.06,0.06,0.04
humidite_pct,0.03,-0.04,-0.04,-0.01,1.00,0.99,-0.00,-0.00,-0.03,0.03
rayonnement_solaire_wm2,0.03,-0.04,-0.04,-0.02,0.99,1.00,-0.00,-0.00,-0.03,0.03
vitesse_vent_kmh,-0.04,0.01,0.02,0.02,-0.00,-0.00,1.00,-0.01,-0.01,-0.02
surface_m2,0.66,0.61,0.59,0.06,-0.00,-0.00,-0.01,1.00,0.46,-0.00
nb_occupants_moyen,0.36,0.30,0.29,0.06,-0.03,-0.03,-0.01,0.46,1.00,0.03
annee_construction,-0.18,-0.08,-0.14,0.04,0.03,0.03,-0.02,-0.00,0.03,1.00


---

## Identifier les correlations significatives (>0.5 ou <-0.5)

In [7]:
corr_sig = (
    matrice_corr.stack()
    .reset_index()
    .rename(columns={"level_0":"var1","level_1":"var2",0:"correlation"})
)

corr_sig = corr_sig[
    (corr_sig["var1"] != corr_sig["var2"]) &
    (corr_sig["correlation"].abs() >= 0.5)
].sort_values("correlation", ascending=False)

corr_sig


,var1,var2,correlation
45,humidite_pct,rayonnement_solaire_wm2,0.99
54,rayonnement_solaire_wm2,humidite_pct,0.99
70,surface_m2,eau,0.66
7,eau,surface_m2,0.66
71,surface_m2,electricite,0.61
17,electricite,surface_m2,0.61
27,gaz,surface_m2,0.59
72,surface_m2,gaz,0.59


---

## Analyser l'impact de la temperature sur la consommation de chauffage

In [8]:
impact_temp_chauffage = df_corr[["gaz","temperature_c"]].corr()

impact_temp_chauffage


,gaz,temperature_c
gaz,1.000000,-0.098947
temperature_c,-0.098947,1.000000


corrélation négative =>  comportement chauffage réaliste (quand la température monte, le chauffage baisse)

---

## Etudier l'effet du rayonnement solaire sur la consommation electrique

In [9]:
impact_solaire_elec = df_corr[["electricite","rayonnement_solaire_wm2"]].corr()

impact_solaire_elec

,electricite,rayonnement_solaire_wm2
electricite,1.00000,-0.03947
rayonnement_solaire_wm2,-0.03947,1.00000


Lecture :

corrélation négative => lumière naturelle alors moins d'éclairage

---

## Enregistrement de la matrice de corelation

In [10]:
matrice_corr.to_csv(OUT_MATR_COR_CSV)
print(f"Matrice de corrélation enregistrée dans : '{OUT_MATR_COR_CSV}'")

Matrice de corrélation enregistrée dans : 'C:\Users\joel\Downloads\ecf_energie\output\07_matrice_correlation.csv'


---

## Enregistrement de la synthese des insights (format markdown)

In [11]:
text_synthese = """
# Synthèse des insights – Analyse énergétique des bâtiments

## Corrélations principales

### Corrélations significatives

* Certaines relations dépassent |0.5|, notamment entre variables structurelles (surface vs consommation totale).
* Les corrélations fortes restent dépendantes du type d’énergie et du contexte d’usage.

---

## Impact de la température sur le chauffage

* Une corrélation négative est observée entre température extérieure et consommation de gaz.
* La relation reste faible sur l’ensemble des données, probablement à cause :

  * du mélange de types de bâtiments,
  * des usages non liés au chauffage,

---

## Effet du rayonnement solaire sur la consommation électrique

* Le rayonnement solaire peut réduire la consommation liée à l’éclairage naturel.
* Dans certains cas, une augmentation peut être observée si la climatisation est utilisée.
* L’effet dépend fortement de la saison et du type de bâtiment.

---

## Conclusion générale

* Les variables météo influencent la consommation, mais leur impact doit être analysé par saison et par énergie pour être pleinement interprété.
* L’identification des bâtiments atypiques constitue un levier prioritaire pour des actions d’efficacité énergétique.

"""

# Sauvegarder le rapport synthese des insight
with open(f"{OUT_RAP_INSIGHT_MD}", 'w', encoding='utf-8') as f:
    f.write(text_synthese)

print(f"Rapport de synthese des insight sauvegardé : {OUT_RAP_INSIGHT_MD}")

Rapport de synthese des insight sauvegardé : C:\Users\joel\Downloads\ecf_energie\output\07_rapport_insight.md


---

## Libérer la mémoire (Optionnel) 

In [12]:
del df_conso
del df_corr
del matrice_corr
del impact_temp_chauffage
del impact_solaire_elec

---

## (Optionnel) enregistrement dans un fichier temporaire du temps d'execution + ressources pour utilisation ultérieure (dans le script run_pipeline_hybride.py ou autres)

In [13]:
temps_execution_07 = time.time() - start_time_07
temps_resources = f"""
    Date : {datetime.now().strftime("%d/%m/%Y %H:%M:%S")} (FR)

    temps_exec_sec={temps_execution_07:.2f}
    ram_gb={ram_available_07:.2f}
    cpu_pct={cpu_available_pct_07:.2f}
    logi_cores={available_logical_07:.1f}
    physi_cores={available_physical_07:.1f}
"""

# Ecrire des données du temps d'execution + ressources dans le fichier TMP_FILE_CSV
TMP_FILE_CSV.write_text(temps_resources, encoding="utf-8")

137